# Recurrent Neural Networks and Language Models

You guys probably very excited about ChatGPT.  In today class, we will be implementing a very simple language model, which is basically what ChatGPT is, but with a simple LSTM.  You will be surprised that it is not so difficult at all.

Paper that we base on is *Regularizing and Optimizing LSTM Language Models*, https://arxiv.org/abs/1708.02182

In [1]:
# for assignment this week

# ------------------
# like github copilot 
# autocomplete

# editor
# import numpy as np
#              as npy


# language modeling -> generate jokes, stories


# "Text"@"Chapter"@"Book"


- Self supervised learning -> no labels needed for Language Modeling
- target sentence, prompt

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext, math
from tqdm import tqdm

In [2]:
!nvidia-smi

Sun Mar 12 12:40:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:84:00.0 Off |                  N/A |
| 24%   33C    P8    23W / 250W |  10962MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [4]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# torch.cuda.get_device_name(0)

cuda:2


## 1. Load data - Wiki Text

We will be using wikitext which contains a large corpus of text, perfect for language modeling task.  This time, we will use the `datasets` library from HuggingFace to load.

In [5]:
import pandas as pd #Importing Pandas package

Bookfile=[] #Empty "Book" list - Prepare for loop

# Loops through importing 7 HP text files - Book 1 creates table, Books 2-7 append to Book 1 table
for i in range(1,8): 
    Bookfile.append('HPBook'+str(i)+'.txt')
    FileLoc = "books/{}".format(Bookfile[i-1])
    if i == 1:
        hp_df = pd.read_csv(FileLoc, sep="@")
    else:
        hp_df2 = pd.read_csv(FileLoc, sep="@")
        hp_df = pd.concat([hp_df, hp_df2])

In [6]:
hp_df

,Text,Chapter,Book
0,"THE BOY WHO LIVED Mr. and Mrs. Dursley, of nu...",1,1
1,THE VANISHING GLASS Nearly ten years had pass...,2,1
2,THE LETTERS FROM NO ONE The escape of the Bra...,3,1
3,THE KEEPER OF THE KEYS BOOM. They knocked aga...,4,1
4,DIAGON ALLEY Harry woke early the next mornin...,5,1
...,...,...,...
32,"Harry remained kneeling at Snape's side, simpl...",33,7
33,"Finally, the truth. Lying with his face presse...",34,7
34,"He lay facedown, listening to the silence. He ...",35,7
35,He was flying facedown on the ground again. Th...,36,7


In [7]:
hp_df.head()

,Text,Chapter,Book
0,"THE BOY WHO LIVED Mr. and Mrs. Dursley, of nu...",1,1
1,THE VANISHING GLASS Nearly ten years had pass...,2,1
2,THE LETTERS FROM NO ONE The escape of the Bra...,3,1
3,THE KEEPER OF THE KEYS BOOM. They knocked aga...,4,1
4,DIAGON ALLEY Harry woke early the next mornin...,5,1


In [8]:
hp_df.Text

0     THE BOY WHO LIVED  Mr. and Mrs. Dursley, of nu...
1     THE VANISHING GLASS  Nearly ten years had pass...
2     THE LETTERS FROM NO ONE  The escape of the Bra...
3     THE KEEPER OF THE KEYS  BOOM. They knocked aga...
4     DIAGON ALLEY  Harry woke early the next mornin...
                            ...                        
32    Harry remained kneeling at Snape's side, simpl...
33    Finally, the truth. Lying with his face presse...
34    He lay facedown, listening to the silence. He ...
35    He was flying facedown on the ground again. Th...
36    Autumn seemed to arrive suddenly that year. Th...
Name: Text, Length: 200, dtype: object

In [9]:
len(hp_df)

200

## For future Reference -> REGEX

^Fran -> ^ start

$gun -> Sagun -> end

\d -> numerical digit

\s -> whitespace

\w -> any letter lower or uppercase,anydigit or underscore

{7} -> number of occurences

[aeiou]{2} -> 2 times aeiou occcurs


In [10]:
text =[]
for x in hp_df.Text[0]:
    text.append(x)

In [11]:
# print(text)

In [12]:
import re
s = """Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of .9 it isn't."""
m = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', s)
for i in m:
    print(i)

Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it.
Did he mind?
Adam Jones Jr. thinks he didn't.
In any case, this isn't true...
Well, with a probability of .9 it isn't.


In [14]:

all_books=""
for x,y in enumerate(hp_df.Text):
    all_books = all_books+y

In [15]:
# all_books

In [16]:
# https://stackoverflow.com/questions/38831457/regex-splitting-strings-at-full-stops-unless-its-part-of-an-honorific#:~:text=l%20%3D%20set(%5B%27Mr.%27%2C%20%27Mrs.%27%2C%20%27Ms.%27%2C%20%27Dr.%27%2C%20%27Prof.%27%2C%20%27Rev.%27%2C%20%27Capt.%27%2C%20%27Lt.%2DCol.%27%2C%20%0A%27Col.%27%2C%20%27Lt.%2DCmdr.%27%2C%20%27The%20Hon.%27%2C%20%27Cmdr.%27%2C%20%27Flt.%20Lt.%27%2C%20%27Brgdr.%27%2C%20%27Wng.%20Cmdr.%27%2C%20%0A%27Group%20Capt.%27%20%2C%27Rt.%27%2C%20%27Maj.%2DGen.%27%2C%20%27Rear%20Admrl.%27%2C%20%27Esq.%27%2C%20%27Mx%27%2C%20%27Adv%27%2C%20%27Jr.%27%5D%20)

l = set(['Mr.', 'Mrs.', 'Ms.', 'Dr.', 'Prof.', 'Rev.', 'Capt.', 'Lt.-Col.', 
'Col.', 'Lt.-Cmdr.', 'The Hon.', 'Cmdr.', 'Flt. Lt.', 'Brgdr.', 'Wng. Cmdr.', 
'Group Capt.' ,'Rt.', 'Maj.-Gen.', 'Rear Admrl.', 'Esq.', 'Mx', 'Adv', 'Jr.'])

final_text = (''.join([w + '\n' if '.' in w and w not in l else w + ' ' for w in all_books.split(' ')]).rstrip())

In [17]:
# hp_df_fin = pd.DataFrame(columns=['sentence'])

data_df = pd.DataFrame([sentence for sentence in final_text.split('\n') if sentence],
                       columns=['sentences'])

In [18]:
data_df.shape

(75264, 1)

In [19]:
data_df.head()

,sentences
0,"THE BOY WHO LIVED Mr. and Mrs. Dursley, of nu..."
1,They were the last people you'd expect to be i...
2,Mr. Dursley was the director of a firm called...
3,"He was a big, beefy man with hardly any neck, ..."
4,Mrs. Dursley was thin and blonde and had nearl...


In [20]:
data_df.sentences[0]

'THE BOY WHO LIVED  Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.'

In [21]:
total_rows = data_df.shape[0]
train_idx = int(70/100 * total_rows)
val_idx = int(20/100 * total_rows)
test_idx = int(10/100 * total_rows)

In [22]:
train_idx,val_idx,test_idx

(52684, 15052, 7526)

In [23]:
train_dataset = data_df[0:train_idx]
val_dataset = data_df[train_idx:train_idx+val_idx]
test_dataset = data_df[train_idx+val_idx:train_idx+val_idx+test_idx]

In [24]:
train_dataset.shape,val_dataset.shape,test_dataset.shape

((52684, 1), (15052, 1), (7526, 1))

In [25]:
from datasets import Dataset

# dataset = Dataset.from_pandas(hp_df)
train_dataset =Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)
test_dataset =Dataset.from_pandas(test_dataset)

In [26]:
train_dataset

Dataset({
    features: ['sentences'],
    num_rows: 52684
})

## 2. Preprocessing

### Tokenizing

Simply tokenize the given text to tokens.

In [27]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['sentences'])}  


tokenized_train_dataset = train_dataset.map(tokenize_data, remove_columns=['sentences'], fn_kwargs={'tokenizer': tokenizer})
tokenized_val_dataset = val_dataset.map(tokenize_data, remove_columns=['sentences'], fn_kwargs={'tokenizer': tokenizer})
tokenized_test_dataset = test_dataset.map(tokenize_data, remove_columns=['sentences'], fn_kwargs={'tokenizer': tokenizer})


Map:   0%|          | 0/52684 [00:00<?, ? examples/s]

Map:   0%|          | 0/15052 [00:00<?, ? examples/s]

Map:   0%|          | 0/7526 [00:00<?, ? examples/s]

In [28]:
tokenized_train_dataset

Dataset({
    features: ['tokens'],
    num_rows: 52684
})

### Numericalizing

We will tell torchtext to add any word that has occurred at least three times in the dataset to the vocabulary because otherwise it would be too big.  Also we shall make sure to add `unk` and `eos`.

In [29]:
## numericalizing
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_train_dataset['tokens'], 
min_freq=3) 
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>'])   
print(len(vocab))                         
print(vocab.get_itos()[:50])       

10117
['<unk>', '<eos>', '.', ',', "'", 'the', 'to', 'and', 'of', 'a', 'he', '\\', 'harry', 'was', 'said', 's', 'his', 'it', 'you', 'in', 'i', '?', 'had', 'that', 'at', 't', 'on', '!', 'as', 'him', 'they', 'with', 'ron', '-', 'for', 'her', 'but', 'hermione', 'she', 'up', 'out', 'not', 'be', 'were', 'all', 'them', 'from', 'what', 'have', 'we']


## 3. Prepare the batch loader

### Prepare data

Given "Chaky loves eating at AIT", and "I really love deep learning", and given batch size = 3, we will get three batches of data "Chaky loves eating at", "AIT `<eos>` I really", "love deep learning `<eos>`".  

In [30]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example['tokens']:         
            #appends eos so we know it ends....so model learn how to end...                             
            tokens = example['tokens'].append('<eos>')   
            #numericalize          
            tokens = [vocab[token] for token in example['tokens']] 
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data


In [31]:
batch_size = 128

train_data = get_data(tokenized_train_dataset, vocab, batch_size)

In [32]:
valid_data = get_data(tokenized_val_dataset, vocab, batch_size)
test_data  = get_data(tokenized_train_dataset, vocab, batch_size)

In [33]:
train_data.shape
# batch size, number of batches
# 16k of 128 batches

torch.Size([128, 7718])

## 4. Modeling Transformer

- batch_first -> If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False

- NER -> use output
- POS tagging -> output
- Sentiment analysis -> mean of all hidden layers-  torch.mean output in 1
                     -> can use both hidden and output

In [40]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers, dropout_rate):   
        super().__init__()
        self.hid_dim = hid_dim
        self.num_layers = num_layers
        #create embeddings
        self.embedding = nn.Embedding(vocab_size,emb_dim)
        #batch is 
        self.LSTM = nn.LSTM(emb_dim,hid_dim,num_layers=num_layers, dropout=dropout_rate,
                           batch_first=True)
        self.dropout= nn.Dropout(dropout_rate)
        
        self.fc = nn.Linear(hid_dim,vocab_size)
        
    def init_hidden(self,batch_size, devide):
        #this function will in the beginning of the epoch
        hidden = torch.zeros(self.num_layers,batch_size,self.hid_dim).to(device)
        cell = torch.zeros(self.num_layers,batch_size,self.hid_dim).to(device)
        
        #return as tuple
        return hidden,cell
    
    def detach_hidden(self,hidden):
        hidden, cell = hidden
        #detaches from cpu/gpu
        #remove from gradients cause it doens't make sense to use gradients for hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell
        
    def forward(self, src, hidden):
        #src ; batchsize, seq_length
        
        #embed
        embed = self.embedding(src)
        #batch_size,seq_len,emb_dim
        
        #now send this to the LSTM
        #hidden has -> hidden state, cell state
        #we want to put hidden here, cause we want to reset hidden
        #we always reset cause we don't learn hidden
        output,hidden = self.LSTM(embed,hidden)
        #output: batch_size,seq_len,hid_dim
        #hidden: num_layer,batch_size,hid_dim
        
        output  = self.dropout(output)
        
        prediction = self.fc(output)
        #we use output for prediction
        #batchsize, seq_len, vocab_size
        
        return prediction,hidden
        
        

## 5. Training 

Follows very basic procedure.  One note is that some of the sequences that will be fed to the model may involve parts from different sequences in the original dataset or be a subset of one (depending on the decoding length). For this reason we will reset the hidden state every epoch, this is like assuming that the next batch of sequences is probably always a follow up on the previous in the original dataset.

In [64]:
# vocab_size, emb_dim, hid_dim, num_layers, dro

In [44]:
vocab_size = len(vocab)
emb_dim = 1024                # 400 in the paper
hid_dim = 1024                # 1150 in the paper
num_layers = 2                # 3 in the paper
dropout_rate = 0.65              
lr = 1e-3                     

In [46]:
model = LSTMLanguageModel(vocab_size, emb_dim, hid_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 37,523,333 trainable parameters


In [47]:
def get_batch(data, seq_len, idx):
    #this data is fromget_data
    #
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [48]:
# model = model
#  data = raw data
# optimizer = Adam
# criterion = Loss function
# batch size = size of the batch
# sequence length = batch ma bhako 
# clip = precents gradient explosion
# device - the device

def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [49]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [50]:
n_epochs = 10
seq_len  = 50
clip    = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):
    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-harry_potter_lstm_lm.pt')

    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 200.350
	Valid Perplexity: 121.904


	Train Perplexity: 87.454
	Valid Perplexity: 93.392


	Train Perplexity: 68.660
	Valid Perplexity: 82.150


	Train Perplexity: 59.073
	Valid Perplexity: 76.360


	Train Perplexity: 52.827
	Valid Perplexity: 72.409


	Train Perplexity: 48.248
	Valid Perplexity: 69.993


	Train Perplexity: 44.578
	Valid Perplexity: 68.643


	Train Perplexity: 41.726
	Valid Perplexity: 67.567


	Train Perplexity: 39.173
	Valid Perplexity: 67.440


	Train Perplexity: 36.975
	Valid Perplexity: 66.850


## 6. Testing

In [51]:
model.load_state_dict(torch.load('best-val-harry_potter_lstm_lm.pt',  map_location=device))
test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
print(f'Test Perplexity: {math.exp(test_loss):.3f}')

Test Perplexity: 26.165


## 7. Real-world inference

Here we take the prompt, tokenize, encode and feed it into the model to get the predictions.  We then apply softmax while specifying that we want the output due to the last word in the sequence which represents the prediction for the next word.  We divide the logits by a temperature value to alter the model’s confidence by adjusting the softmax probability distribution.

Once we have the Softmax distribution, we randomly sample it to make our prediction on the next word. If we get <unk> then we give that another try.  Once we get <eos> we stop predicting.
    
We decode the prediction back to strings last lines.

In [52]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

- temperature 
- softmax -> argmax -> temp is 1
- higher probability gets more chance in softmax hill jasto graph huncha softmax ko
- temperature is dividing this probability
- end/seq_len bhetesi end huncha sentence generate 

In [59]:
prompt = 'hogwarts'
max_seq_len = 30
seed = 0

#smaller the temperature, more diverse tokens but comes 
#with a tradeoff of less-make-sense sentence
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
hogwarts was still very difficult .

0.7
hogwarts discord had been attacked by the fat lady , and because he was scared of the school .

0.75
hogwarts discord had been attacked by the fat lady , and because he was scared of the dangers of fighting himself in the distance , harry felt it if it was

0.8
hogwarts discord had been attacked by the fat lady , and because none of the dursleys were spending more fighting the confidence of jupiter ' s memories , if it was

1.0
hogwarts discord had patched most of the fat lady in a because in hearing the dursleys ' statute of fighting investigate , but in two years ago , if it was



# References

- https://github.com/ErikaJacobs/Harry-Potter-Text-Mining/blob/master/Notebooks/HP_Text_Mining%20-%20Exploratory%20Data%20Analysis.ipynb